# Fight Prediction

We create a simple baseline model to predict the winner of UFC's fight based on a dataset with about 5000 fight and over 100 feature such as stance, rank, time, finish, betting odds, striking percentage, number of takedown attempt, ...

In [1]:
# Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
pd.set_option("display.max_columns", None)

Looking at the data

In [3]:
df = pd.read_csv("data/ufc-master.csv")
df

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,title_bout,weight_class,gender,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_longest_win_streak,B_losses,B_total_rounds_fought,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_landed,R_avg_TD_pct,R_longest_win_streak,R_losses,R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,B_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,B_match_weightclass_rank,R_match_weightclass_rank,R_Women's Flyweight_rank,R_Women's Featherweight_rank,R_Women's Strawweight_rank,R_Women's Bantamweight_rank,R_Heavyweight_rank,R_Light Heavyweight_rank,R_Middleweight_rank,R_Welterweight_rank,R_Lightweight_rank,R_Featherweight_rank,R_Bantamweight_rank,R_Flyweight_rank,R_Pound-for-Pound_rank,B_Women's Flyweight_rank,B_Women's Featherweight_rank,B_Women's Strawweight_rank,B_Women's Bantamweight_rank,B_Heavyweight_rank,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
0,Thiago Santos,Johnny Walker,-150.0,130,66.666667,130.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,False,Light Heavyweight,MALE,5,0,1,0,3.420000,0.59,0.700000,0.240000,1.00,4,2,11,0,0,0,1,4,0,0,5,Orthodox,198.12,208.28,205,3,0,0,3.950000,0.480000,0.100000,0.610000,0.360,4,8,43,1,0,0,2,11,0,0,13,Orthodox,187.96,193.04,205,37,29,-3,1,0,-8,-6,-32,-1,-7,0,10.16,15.24,-8,-0.530000,0.600000,-0.370000,1,1,10.0,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red,U-DEC,NaN,5.0,5:00,1500.0,800.0,900.0,2000.0,1600.0,-110.0,175.0
1,Alex Oliveira,Niko Price,170.0,-200,170.000000,50.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Welterweight,MALE,3,2,0,0,5.160000,0.42,0.800000,0.790000,0.22,2,5,24,0,0,0,0,4,2,0,6,Orthodox,182.88,193.04,170,2,0,0,2.970000,0.510000,0.500000,2.270000,0.370,4,8,44,0,0,1,3,4,3,0,11,Orthodox,180.34,193.04,170,33,32,0,0,-2,-5,-3,-20,0,0,-1,2.54,0.00,-1,2.190000,0.300000,-1.480000,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,U-DEC,NaN,3.0,5:00,900.0,450.0,350.0,700.0,1100.0,550.0,120.0
2,Misha Cirkunov,Krzysztof Jotko,110.0,-130,110.000000,76.923077,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Middleweight,MALE,3,1,0,0,2.920000,0.41,0.100000,1.150000,0.34,5,5,38,0,0,2,6,1,0,0,9,Southpaw,185.42,195.58,185,1,0,0,3.770000,0.490000,1.700000,4.480000,0.440,4,4,13,0,0,0,0,1,5,0,6,Orthodox,190.50,195.58,205,34,32,0,0,1,3,1,25,0,0,-5,-5.08,0.00,-2,-0.850000,-1.600000,-3.330000,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,S-DEC,NaN,3.0,5:00,900.0,550.0,275.0,275.0,1400.0,600.0,185.0
3,Alexander Hernandez,Mike Breeden,-675.0,475,14.814815,475.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,False,Lightweight,MALE,3,1,0,0,4.040000,0.34,0.000000,0.000000,0.00,0,1,3,0,0,0,0,0,0,0,0,Orthodox,177.80,177.80,155,1,0,0,3.790000,0.360000,0.000000,1.570000,0.350,2,3,15,0,0,0,2,2,0,0,4,Orthodox,17

Let's first quickly simple the dataset a bit. We have the ranking in each division for each fighter (12 division, 2 fighter, so 24 columns per row). Since a fighter mostly fight in a single division, most of those columns will have a value of NaN. Let's aggregate and take the average of those values and have only one ranking column per fighter.

In [4]:
#Aggregate the rank into a single columns 

def get_rank_r(row):
    """Calculate the average ranking across all divisions for fighter Red"""
    ranks = row["R_Women's Flyweight_rank"], row["R_Women's Featherweight_rank"], row["R_Women's Strawweight_rank"], row["R_Women's Bantamweight_rank"], row["R_Heavyweight_rank"], row["R_Light Heavyweight_rank"], row["R_Middleweight_rank"], row["R_Welterweight_rank"], row["R_Lightweight_rank"], row["R_Featherweight_rank"], row["R_Bantamweight_rank"], row["R_Flyweight_rank"], row["R_Pound-for-Pound_rank"], row["R_match_weightclass_rank"]
    ranks = list(ranks)
    nan_bool = np.isnan(ranks)
    weights = [1 if ~n else 0 for n in nan_bool]
    masked_data = np.ma.masked_array(ranks, np.isnan(ranks))
    if sum(weights):
        return np.average(masked_data, weights=weights, axis=0)
    if not sum(weights):
        return np.nan
def get_rank_b(row):
    """Calculate the average ranking across all divisions for fighter Blue"""
    ranks = row["B_Women's Flyweight_rank"], row["B_Women's Featherweight_rank"], row["B_Women's Strawweight_rank"], row["B_Women's Bantamweight_rank"], row["B_Heavyweight_rank"], row["B_Light Heavyweight_rank"], row["B_Middleweight_rank"], row["B_Welterweight_rank"], row["B_Lightweight_rank"], row["B_Featherweight_rank"], row["B_Bantamweight_rank"], row["B_Flyweight_rank"], row["B_Pound-for-Pound_rank"], row["B_match_weightclass_rank"]
    ranks = list(ranks)
    nan_bool = np.isnan(ranks)
    weights = [1 if ~n else 0 for n in nan_bool]
    masked_data = np.ma.masked_array(ranks, np.isnan(ranks))
    if sum(weights):
        return np.average(masked_data, weights=weights, axis=0)
    if not sum(weights):
        return np.nan
    

# Add new ranking columns
df["R_rank"] = df.apply(lambda row: get_rank_r(row), axis=1)
df["B_rank"] = df.apply(lambda row: get_rank_b(row), axis=1)


In [5]:
# Drop all those now useless columns
print(df.shape)
df = df.drop(["R_Women's Flyweight_rank","R_Women's Featherweight_rank","R_Women's Strawweight_rank","R_Women's Bantamweight_rank","R_Heavyweight_rank","R_Light Heavyweight_rank","R_Middleweight_rank","R_Welterweight_rank","R_Lightweight_rank","R_Featherweight_rank","R_Bantamweight_rank","R_Flyweight_rank", "R_Pound-for-Pound_rank", "R_match_weightclass_rank"], axis=1)
df = df.drop(["B_Women's Flyweight_rank","B_Women's Featherweight_rank","B_Women's Strawweight_rank","B_Women's Bantamweight_rank","B_Heavyweight_rank","B_Light Heavyweight_rank","B_Middleweight_rank","B_Welterweight_rank","B_Lightweight_rank","B_Featherweight_rank","B_Bantamweight_rank","B_Flyweight_rank", "B_Pound-for-Pound_rank", "B_match_weightclass_rank"], axis=1)

print(df.shape)
df

(4896, 121)
(4896, 93)


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,title_bout,weight_class,gender,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_longest_win_streak,B_losses,B_total_rounds_fought,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_landed,R_avg_TD_pct,R_longest_win_streak,R_losses,R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,B_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,better_rank,finish,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,R_rank,B_rank
0,Thiago Santos,Johnny Walker,-150.0,130,66.666667,130.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,False,Light Heavyweight,MALE,5,0,1,0,3.420000,0.59,0.700000,0.240000,1.00,4,2,11,0,0,0,1,4,0,0,5,Orthodox,198.12,208.28,205,3,0,0,3.950000,0.480000,0.100000,0.610000,0.360,4,8,43,1,0,0,2,11,0,0,13,Orthodox,187.96,193.04,205,37,29,-3,1,0,-8,-6,-32,-1,-7,0,10.16,15.24,-8,-0.530000,0.600000,-0.370000,1,1,Red,U-DEC,NaN,5.0,5:00,1500.0,800.0,900.0,2000.0,1600.0,-110.0,175.0,5.0,10.0
1,Alex Oliveira,Niko Price,170.0,-200,170.000000,50.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Welterweight,MALE,3,2,0,0,5.160000,0.42,0.800000,0.790000,0.22,2,5,24,0,0,0,0,4,2,0,6,Orthodox,182.88,193.04,170,2,0,0,2.970000,0.510000,0.500000,2.270000,0.370,4,8,44,0,0,1,3,4,3,0,11,Orthodox,180.34,193.04,170,33,32,0,0,-2,-5,-3,-20,0,0,-1,2.54,0.00,-1,2.190000,0.300000,-1.480000,1,1,neither,U-DEC,NaN,3.0,5:00,900.0,450.0,350.0,700.0,1100.0,550.0,120.0,NaN,NaN
2,Misha Cirkunov,Krzysztof Jotko,110.0,-130,110.000000,76.923077,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Middleweight,MALE,3,1,0,0,2.920000,0.41,0.100000,1.150000,0.34,5,5,38,0,0,2,6,1,0,0,9,Southpaw,185.42,195.58,185,1,0,0,3.770000,0.490000,1.700000,4.480000,0.440,4,4,13,0,0,0,0,1,5,0,6,Orthodox,190.50,195.58,205,34,32,0,0,1,3,1,25,0,0,-5,-5.08,0.00,-2,-0.850000,-1.600000,-3.330000,1,1,neither,S-DEC,NaN,3.0,5:00,900.0,550.0,275.0,275.0,1400.0,600.0,185.0,13.0,NaN
3,Alexander Hernandez,Mike Breeden,-675.0,475,14.814815,475.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,False,Lightweight,MALE,3,1,0,0,4.040000,0.34,0.000000,0.000000,0.00,0,1,3,0,0,0,0,0,0,0,0,Orthodox,177.80,177.80,155,1,0,0,3.790000,0.360000,0.000000,1.570000,0.350,2,3,15,0,0,0,2,2,0,0,4,Orthodox,175.26,182.88,155,29,32,0,0,-2,-4,-2,-12,0,-2,0,2.54,-5.08,3,0.250000,0.000000,-1.570000,1,1,neither,KO/TKO,Punch,1.0,1:20,80.0,175.0,900.0,500.0,3500.0,110.0,1100.0,NaN,NaN
4,Joe Solecki,Jared Gordon,-135.0,115,74.074074,115.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,False,Lightweight,MALE,3,0,2,0,5.220000,0.56,0.000000,2.540000,0.39,2,3,19,0,0,0,4,1,0,0,5,Orthodox,175.26,172.72,145,0,4,0,2.640000,0.620000,0.600000,2.850000,0.520,4,0,8,0,0,0,2,0,2,0,4,Orthodox,175.26,177.80,155,28,33,0,-2,-2,1,3,11,0,1,-2,0.00,-5.08,5,2.580000,-0.600000,-0.310000,1,1,neither,S-DEC,NaN,3.0,5:00,900.0,165.0,200.0,400.0,1200.0,900.0,600.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

We can separate our feature columns into categorical (date, fighter stance, finishing detail,...) and numerical (rank, time, odds, strike percentage,...)

In [6]:
categorical_features = [col for col in df.columns if df[col].dtypes == 'object']
numeric_features = [col for col in df.columns if col not in categorical_features]
print(f"Categorical feature: {categorical_features}")
print(f"Numerical feature: {numeric_features}")
#TODO check features (title_bout)


Categorical feature: ['R_fighter', 'B_fighter', 'date', 'location', 'country', 'Winner', 'weight_class', 'gender', 'B_Stance', 'R_Stance', 'better_rank', 'finish', 'finish_details', 'finish_round_time']
Numerical feature: ['R_odds', 'B_odds', 'R_ev', 'B_ev', 'title_bout', 'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_De

This is a classification problem, we want to predict the winner so we get our label values:

In [7]:
## Get label
label = df.Winner
label = [1 if win == 'Red' else 0 if win == 'Blue' else np.nan for win in label]

We split our data into training and testing set

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(df, label, random_state = 2, test_size = 0.3)

## Building a baseline model

We first create a baseline model to compare a better model's performance later on


Imputation. We simply use the most frequent value to impute missing data

In [9]:
enc = LabelEncoder()
for i in df[categorical_features]:
    #using astype(str) to avoid columns with 'float and str' to throw errors
    df[i] = enc.fit_transform(df[i].astype(str))

#Numerical feature
imp = SimpleImputer(strategy='most_frequent')
imp.fit(X_train[numeric_features])
imp.transform(X_train[numeric_features]).shape
X_valid[numeric_features] = imp.transform(X_valid[numeric_features])

#Categorical feature
cat_imp = SimpleImputer(strategy = 'most_frequent')
cat_imp.fit(X_train[categorical_features])
X_train[categorical_features] = cat_imp.transform(X_train[categorical_features])
X_valid[categorical_features] = cat_imp.transform(X_valid[categorical_features])
X_train.sample(10)


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,title_bout,weight_class,gender,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_longest_win_streak,B_losses,B_total_rounds_fought,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_landed,R_avg_TD_pct,R_longest_win_streak,R_losses,R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,B_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,better_rank,finish,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,R_rank,B_rank
4049,Daniel Sarafian,CB Dollaway,-140.0,130,71.428571,130.000000,1/19/2013,"Sao Paulo, Brazil",Brazil,Blue,False,Middleweight,MALE,3,0,1,0,12.800000,0.444000,0.900000,2.100000,0.530000,3,4,17,1,0,0,3,1,2,0,6,Orthodox,187.96,193.04,185,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,Orthodox,175.26,185.42,185,30,29,0,1,3,6,-4,17,1,1,2,12.70,7.62,1,12.800000,0.900000,2.100000,0,1,neither,S-DEC,Punch,3.0,5:00,900.0,695.0,210.0,290.0,900.0,260.0,1167.0,NaN,NaN
4777,Nick Osipczak,Greg Soto,-200.0,170,50.000000,170.000000,8/28/2010,"Boston, Massachusetts, USA",USA,Blue,False,Welterweight,MALE,3,1,0,0,21.000000,0.530000,0.000000,0.000000,0.000000,0,1,3,0,0,0,0,0,0,0,0,Orthodox,180.34,185.42,170,1,0,0,29.333333,0.530000,1.000000,1.000000,0.500000,2,1,7,0,0,0,0,1,1,0,2,Orthodox,187.96,193.04,170,25,24,0,0,-2,-2,0,-4,0,-1,-1,-7.62,-7.62,1,-8.333333,-1.000000,-1.000000,0,1,neither,U-DEC,Punch,3.0,5:00,900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3122,Masio Fullen,Alex Torres,105.0,-115,105.000000,86.956522,2/28/2015,"Los Angeles, California, USA",USA,Red,False,Featherweight,MALE,3,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,Orthodox,177.80,177.80,145,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,Orthodox,177.80,177.80,145,28,26,0,0,0,0,0,0,0,0,0,0.00,0.00,2,0.000000,0.000000,0.000000,0,1,neither,S-DEC,Punch,3.0,5:00,900.0,415.0,295.0,443.0,664.0,330.0,390.0,NaN,NaN
3255,Marco Beltran,Marlon Vera,205.0,-225,205.000000,44.444444,11/15/2014,"Mexico City, Distrito Federal, Mexico",Mexico,Red,False,Bantamweight,MALE,3,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,Switch,172.72,177.80,135,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,Orthodox,172.72,175.26,125,28,21,0,0,0,0,0,0,0,0,0,0.00,2.54,7,0.000000,0.000000,0.000000,0,1,neither,U-DEC,Punch,3.0,5:00,900.0,450.0,240.0,815.0,300.0,645.0,253.0,NaN,NaN
4870,Rob Emerson,Nik Lentz,130.0,-150,130.000000,66.666667,3/31/2010,"Charlotte, North Carolina, USA",USA,Blue,False,Lightweight,MALE,3,1,0,0,69.000000,0.415000,2.500000,4.000000,0.550000,1,1,6,0,0,0,1,0,0,0,1,Orthodox,172.72,172.72,155,0,1,0,26.833333,0.426667,0.166667,1.666667,0.338333,2,3,14,0,0,1,1,1,0,0,3,Orthodox,175.26,177.80,155,28,25,-1,-1,-1,-2,2,-8,0,-1,0,-2.54,-5.08,3,42.166667,2.333333,2.333333,0,1,neither,U-DEC,Punch,3.0,5:00,900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4111,Ivan Menjivar,Azamat Gashimov,-215.0,195,46.511628,195.000000,11/17/2012,"Montreal, Quebec, Canada",Canada,Red,False,Bantamweight,MALE,3,0,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,Orthodox,167.64,167.64,125,1,0,0,28.400000,0.428000,0.800000,0.200000,0.200000,3,2,11,0,0,0,1,1,1,0,3,Orthodox,167.64,162.56,145,30,22,1,0,-3,-3,2,-11,0,-1,-1,0.00,5.08,8,-28.400000,-0.800000,-0

In [10]:
base_model = DummyClassifier(random_state=2)
base_model.fit(X_train,y_train)

DummyClassifier(random_state=2)

In [11]:
preds = base_model.predict(X_valid)
print(f"We get an accuracy of {round(accuracy_score(y_valid, preds),3)} for our baseline model, not bad.")


We get an accuracy of 0.587 for our baseline model, not bad.
